In [21]:
open System.Net.Http
open System.Text.Json
open System.Text.Json.Serialization
open System
open System.IO
open System.Net.Http.Headers

In [ ]:
type Credentials() =
    member val username: string = null with get, set 
    member val key: string = null with get,set
      
    static member LoadFrom(path: string): Credentials =
        use reader = new StreamReader(path)
        let json = reader.ReadToEnd()
        JsonSerializer.Deserialize<Credentials>(json)

In [ ]:
let creds = Credentials.LoadFrom("kaggle.json")
creds

In [ ]:
let BaseApiUrl = "https://www.kaggle.com/api/v1/"

type AuthorizedClient = AuthorizedClient of HttpClient

In [ ]:
let CreateAuthorizedClient(auth: Credentials) =
    let authToken =
        sprintf "%s:%s" auth.username auth.key
        |> Text.ASCIIEncoding.ASCII.GetBytes
        |> Convert.ToBase64String

    let client = new HttpClient()
    client.DefaultRequestHeaders.Authorization <- AuthenticationHeaderValue("Basic", authToken)

    AuthorizedClient client

let DownLoadFileAsync (urlPath: string[]) destinationFolder (AuthorizedClient client) =
    let url = sprintf "%sdatasets/download/%s" BaseApiUrl <| String.Join("/", urlPath)
    let filename = urlPath |> Array.last

    async {
        use! stream = client.GetStreamAsync(url) |> Async.AwaitTask
        use fstream = new FileStream(Path.Combine(destinationFolder, filename), FileMode.CreateNew)
        let! _ = stream.CopyToAsync fstream |> Async.AwaitTask
        fstream.Close()
        stream.Close()
    }

In [ ]:
let authClient = CreateAuthorizedClient creds
let (AuthorizedClient client) = authClient

In [ ]:
DownLoadFileAsync [|"selfishgene";"historical-hourly-weather-data.zip"|] "./" authClient
|> Async.RunSynchronously


In [22]:
let DownloadFileAsync 
        (url: string) (destinationFile: string) cancellationToken 
        (report: int64 * float -> unit) (client: HttpClient) =
        async {
            let bufferLength = 4092
            use! response = client.GetAsync(url, HttpCompletionOption.ResponseHeadersRead, cancellationToken)
                            |> Async.AwaitTask
            
            response.EnsureSuccessStatusCode() |> ignore

            let total =
                if response.Content.Headers.ContentLength.HasValue
                then response.Content.Headers.ContentLength.Value
                else -1L
                |> float

            use! contentStream = response.Content.ReadAsStreamAsync() |> Async.AwaitTask
            use fileStream =
                new FileStream(destinationFile, FileMode.Create, FileAccess.Write, FileShare.None, bufferLength, true)

            let mutable totalRead = 0L
            let mutable totalReads = 0L
            let mutable isMoreToRead = true
            let buffer = Array.create bufferLength 0uy

            while isMoreToRead && not cancellationToken.IsCancellationRequested do
                let! read = contentStream.ReadAsync(buffer, 0, bufferLength) |> Async.AwaitTask
                if read > 0 then
                    do! fileStream.WriteAsync(buffer, 0, read) |> Async.AwaitTask
                    totalRead <- totalRead + int64 read
                    totalReads <- totalReads + 1L
                else
                    isMoreToRead <- false

                report (totalRead, float totalRead / total)
        }

In [18]:
open System
open Microsoft.DotNet.Interactive.FSharp
open Microsoft.DotNet.Interactive
open Microsoft.AspNetCore.Html 
open FSharpKernelHelpers 

let progLog (bytesRead : int64,percentage: float) = 
    if int (percentage * 100.) % 10 = 0
    then 
        display(HtmlString("<span style='width: 3em; backroung-color: green; padding: 1em'><span>")) |> ignore
    

In [25]:
DownloadFileAsync "https://i.redd.it/u5pz59lmxocy.png" "./jupyter-test.png" (CancellationToken()) progLog (HttpClient())
|> Async.RunSynchronously

In [24]:
open System
open System.Net.Http
open System.Threading.Tasks
open System.Threading